# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 28 2023 10:44AM,258356,10,0086462295,ISDIN Corporation,Released
1,Feb 28 2023 10:43AM,258355,16,9217096,Sartorius Bioprocess Solutions,Released
2,Feb 28 2023 10:43AM,258355,16,9217098,Sartorius Bioprocess Solutions,Released
3,Feb 28 2023 10:42AM,258354,10,0086459135,ISDIN Corporation,Released
4,Feb 28 2023 10:42AM,258354,10,0086459134,ISDIN Corporation,Released
5,Feb 28 2023 10:42AM,258354,10,0086459145,ISDIN Corporation,Released
6,Feb 28 2023 10:42AM,258354,10,0086459146,ISDIN Corporation,Released
7,Feb 28 2023 10:42AM,258354,10,0086459147,ISDIN Corporation,Released
8,Feb 28 2023 10:42AM,258354,10,0086459149,ISDIN Corporation,Released
9,Feb 28 2023 10:42AM,258354,10,0086459133,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,258352,Released,13
45,258353,Released,18
46,258354,Released,49
47,258355,Released,2
48,258356,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258352,NaN,NaN,13.0
258353,NaN,NaN,18.0
258354,NaN,NaN,49.0
258355,NaN,NaN,2.0
258356,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258225,0.0,1.0,6.0
258227,0.0,0.0,1.0
258234,0.0,0.0,1.0
258245,0.0,1.0,0.0
258251,0.0,1.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258225,0,1,6
258227,0,0,1
258234,0,0,1
258245,0,1,0
258251,0,1,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258225,0,1,6
1,258227,0,0,1
2,258234,0,0,1
3,258245,0,1,0
4,258251,0,1,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258225,,1,6
1,258227,,,1
2,258234,,,1
3,258245,,1,
4,258251,,1,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 28 2023 10:44AM,258356,10,ISDIN Corporation
1,Feb 28 2023 10:43AM,258355,16,Sartorius Bioprocess Solutions
3,Feb 28 2023 10:42AM,258354,10,ISDIN Corporation
52,Feb 28 2023 10:31AM,258353,10,ISDIN Corporation
70,Feb 28 2023 10:30AM,258352,10,Bio-PRF
83,Feb 28 2023 10:15AM,258351,10,ISDIN Corporation
84,Feb 28 2023 10:09AM,258350,10,"Digital Brands, LLC"
89,Feb 28 2023 9:59AM,258349,10,ISDIN Corporation
104,Feb 28 2023 9:54AM,258348,15,"Mizner Bioscience, LLC"
106,Feb 28 2023 9:52AM,258347,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 28 2023 10:44AM,258356,10,ISDIN Corporation,,,1
1,Feb 28 2023 10:43AM,258355,16,Sartorius Bioprocess Solutions,,,2
2,Feb 28 2023 10:42AM,258354,10,ISDIN Corporation,,,49
3,Feb 28 2023 10:31AM,258353,10,ISDIN Corporation,,,18
4,Feb 28 2023 10:30AM,258352,10,Bio-PRF,,,13
5,Feb 28 2023 10:15AM,258351,10,ISDIN Corporation,,,1
6,Feb 28 2023 10:09AM,258350,10,"Digital Brands, LLC",,,5
7,Feb 28 2023 9:59AM,258349,10,ISDIN Corporation,,,15
8,Feb 28 2023 9:54AM,258348,15,"Mizner Bioscience, LLC",,,2
9,Feb 28 2023 9:52AM,258347,10,"Methapharm, Inc.",,,18


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 10:44AM,258356,10,ISDIN Corporation,1,,
1,Feb 28 2023 10:43AM,258355,16,Sartorius Bioprocess Solutions,2,,
2,Feb 28 2023 10:42AM,258354,10,ISDIN Corporation,49,,
3,Feb 28 2023 10:31AM,258353,10,ISDIN Corporation,18,,
4,Feb 28 2023 10:30AM,258352,10,Bio-PRF,13,,
5,Feb 28 2023 10:15AM,258351,10,ISDIN Corporation,1,,
6,Feb 28 2023 10:09AM,258350,10,"Digital Brands, LLC",5,,
7,Feb 28 2023 9:59AM,258349,10,ISDIN Corporation,15,,
8,Feb 28 2023 9:54AM,258348,15,"Mizner Bioscience, LLC",2,,
9,Feb 28 2023 9:52AM,258347,10,"Methapharm, Inc.",18,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 10:44AM,258356,10,ISDIN Corporation,1,,
1,Feb 28 2023 10:43AM,258355,16,Sartorius Bioprocess Solutions,2,,
2,Feb 28 2023 10:42AM,258354,10,ISDIN Corporation,49,,
3,Feb 28 2023 10:31AM,258353,10,ISDIN Corporation,18,,
4,Feb 28 2023 10:30AM,258352,10,Bio-PRF,13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 10:44AM,258356,10,ISDIN Corporation,1.0,NaN,NaN
1,Feb 28 2023 10:43AM,258355,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN
2,Feb 28 2023 10:42AM,258354,10,ISDIN Corporation,49.0,NaN,NaN
3,Feb 28 2023 10:31AM,258353,10,ISDIN Corporation,18.0,NaN,NaN
4,Feb 28 2023 10:30AM,258352,10,Bio-PRF,13.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 10:44AM,258356,10,ISDIN Corporation,1.0,0.0,0.0
1,Feb 28 2023 10:43AM,258355,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0
2,Feb 28 2023 10:42AM,258354,10,ISDIN Corporation,49.0,0.0,0.0
3,Feb 28 2023 10:31AM,258353,10,ISDIN Corporation,18.0,0.0,0.0
4,Feb 28 2023 10:30AM,258352,10,Bio-PRF,13.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5683207,352.0,0.0,0.0
12,516587,1.0,1.0,0.0
15,1549659,79.0,19.0,0.0
16,516684,2.0,1.0,0.0
19,1033278,8.0,21.0,0.0
20,516627,1.0,5.0,18.0
21,516558,1.0,1.0,0.0
22,258338,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5683207,352.0,0.0,0.0
1,12,516587,1.0,1.0,0.0
2,15,1549659,79.0,19.0,0.0
3,16,516684,2.0,1.0,0.0
4,19,1033278,8.0,21.0,0.0
5,20,516627,1.0,5.0,18.0
6,21,516558,1.0,1.0,0.0
7,22,258338,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,352.0,0.0,0.0
1,12,1.0,1.0,0.0
2,15,79.0,19.0,0.0
3,16,2.0,1.0,0.0
4,19,8.0,21.0,0.0
5,20,1.0,5.0,18.0
6,21,1.0,1.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,352.0
1,12,Released,1.0
2,15,Released,79.0
3,16,Released,2.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0
Executing,0.0,1.0,19.0,1.0,21.0,5.0,1.0,1.0
Released,352.0,1.0,79.0,2.0,8.0,1.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0
1,Executing,0.0,1.0,19.0,1.0,21.0,5.0,1.0,1.0
2,Released,352.0,1.0,79.0,2.0,8.0,1.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0
1,Executing,0.0,1.0,19.0,1.0,21.0,5.0,1.0,1.0
2,Released,352.0,1.0,79.0,2.0,8.0,1.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()